ਹੇਠਾਂ ਦਿੱਤੇ ਨੋਟਬੁੱਕ ਚਲਾਉਣ ਲਈ, ਜੇਕਰ ਤੁਸੀਂ ਅਜੇ ਤੱਕ ਨਹੀਂ ਕੀਤਾ, ਤਾਂ ਤੁਹਾਨੂੰ .env ਫਾਈਲ ਦੇ ਅੰਦਰ openai ਕੁੰਜੀ ਨੂੰ `OPENAI_API_KEY` ਵਜੋਂ ਸੈੱਟ ਕਰਨਾ ਲਾਜ਼ਮੀ ਹੈ।


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )

model = 'text-embedding-ada-002'

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

ਅਗਲੇ, ਅਸੀਂ ਐਮਬੈਡਿੰਗ ਇੰਡੈਕਸ ਨੂੰ ਇੱਕ ਪੈਂਡਾਸ ਡਾਟਾਫਰੇਮ ਵਿੱਚ ਲੋਡ ਕਰਨ ਜਾ ਰਹੇ ਹਾਂ। ਐਮਬੈਡਿੰਗ ਇੰਡੈਕਸ ਇੱਕ JSON ਫਾਇਲ ਵਿੱਚ ਸਟੋਰ ਕੀਤਾ ਗਿਆ ਹੈ ਜਿਸਦਾ ਨਾਮ `embedding_index_3m.json` ਹੈ। ਐਮਬੈਡਿੰਗ ਇੰਡੈਕਸ ਵਿੱਚ ਹਰ ਯੂਟਿਊਬ ਟ੍ਰਾਂਸਕ੍ਰਿਪਟ ਲਈ ਐਮਬੈਡਿੰਗਜ਼ ਸ਼ਾਮਲ ਹਨ ਜੋ ਅਕਤੂਬਰ 2023 ਦੇ ਅੰਤ ਤੱਕ ਹਨ।


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

ਅਗਲੇ, ਅਸੀਂ `get_videos` ਨਾਮਕ ਇੱਕ ਫੰਕਸ਼ਨ ਬਣਾਉਣ ਜਾ ਰਹੇ ਹਾਂ ਜੋ ਕਵੈਰੀ ਲਈ ਐਮਬੈਡਿੰਗ ਇੰਡੈਕਸ ਵਿੱਚ ਖੋਜ ਕਰੇਗਾ। ਇਹ ਫੰਕਸ਼ਨ ਉਹ 5 ਸਿਖਰਲੇ ਵੀਡੀਓਜ਼ ਵਾਪਸ ਕਰੇਗਾ ਜੋ ਕਵੈਰੀ ਨਾਲ ਸਭ ਤੋਂ ਵੱਧ ਮਿਲਦੇ-ਜੁਲਦੇ ਹਨ। ਫੰਕਸ਼ਨ ਇਸ ਤਰ੍ਹਾਂ ਕੰਮ ਕਰਦਾ ਹੈ:

1. ਸਭ ਤੋਂ ਪਹਿਲਾਂ, ਐਮਬੈਡਿੰਗ ਇੰਡੈਕਸ ਦੀ ਇੱਕ ਨਕਲ ਬਣਾਈ ਜਾਂਦੀ ਹੈ।
2. ਅਗਲੇ ਕਦਮ ਵਿੱਚ, ਕਵੈਰੀ ਲਈ ਐਮਬੈਡਿੰਗ OpenAI ਐਮਬੈਡਿੰਗ API ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਗਣਨਾ ਕੀਤੀ ਜਾਂਦੀ ਹੈ।
3. ਫਿਰ ਐਮਬੈਡਿੰਗ ਇੰਡੈਕਸ ਵਿੱਚ `similarity` ਨਾਮਕ ਇੱਕ ਨਵਾਂ ਕਾਲਮ ਬਣਾਇਆ ਜਾਂਦਾ ਹੈ। `similarity` ਕਾਲਮ ਵਿੱਚ ਕਵੈਰੀ ਐਮਬੈਡਿੰਗ ਅਤੇ ਹਰ ਵੀਡੀਓ ਸੈਗਮੈਂਟ ਦੀ ਐਮਬੈਡਿੰਗ ਦਰਮਿਆਨ ਕੋਸਾਈਨ ਸਮਾਨਤਾ ਹੁੰਦੀ ਹੈ।
4. ਅਗਲੇ ਕਦਮ ਵਿੱਚ, ਐਮਬੈਡਿੰਗ ਇੰਡੈਕਸ ਨੂੰ `similarity` ਕਾਲਮ ਦੇ ਆਧਾਰ 'ਤੇ ਫਿਲਟਰ ਕੀਤਾ ਜਾਂਦਾ ਹੈ। ਐਮਬੈਡਿੰਗ ਇੰਡੈਕਸ ਨੂੰ ਸਿਰਫ ਉਹ ਵੀਡੀਓਜ਼ ਸ਼ਾਮਲ ਕਰਨ ਲਈ ਫਿਲਟਰ ਕੀਤਾ ਜਾਂਦਾ ਹੈ ਜਿਨ੍ਹਾਂ ਦੀ ਕੋਸਾਈਨ ਸਮਾਨਤਾ 0.75 ਜਾਂ ਇਸ ਤੋਂ ਵੱਧ ਹੁੰਦੀ ਹੈ।
5. ਆਖ਼ਿਰ ਵਿੱਚ, ਐਮਬੈਡਿੰਗ ਇੰਡੈਕਸ ਨੂੰ `similarity` ਕਾਲਮ ਦੇ ਆਧਾਰ 'ਤੇ ਛਾਂਟਿਆ ਜਾਂਦਾ ਹੈ ਅਤੇ ਸਿਖਰਲੇ 5 ਵੀਡੀਓਜ਼ ਵਾਪਸ ਕੀਤੇ ਜਾਂਦੇ ਹਨ।


In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

ਇਹ ਫੰਕਸ਼ਨ ਬਹੁਤ ਸਧਾਰਣ ਹੈ, ਇਹ ਸਿਰਫ ਖੋਜ ਪ੍ਰਸ਼ਨ ਦੇ ਨਤੀਜੇ ਪ੍ਰਿੰਟ ਕਰਦਾ ਹੈ।


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. ਸਭ ਤੋਂ ਪਹਿਲਾਂ, ਐਮਬੈਡਿੰਗ ਇੰਡੈਕਸ ਨੂੰ ਪੈਂਡਾਸ ਡਾਟਾਫਰੇਮ ਵਿੱਚ ਲੋਡ ਕੀਤਾ ਜਾਂਦਾ ਹੈ।
2. ਅਗਲੇ ਕਦਮ ਵਿੱਚ, ਯੂਜ਼ਰ ਨੂੰ ਇੱਕ ਕਵੈਰੀ ਦਰਜ ਕਰਨ ਲਈ ਪ੍ਰੰਪਟ ਕੀਤਾ ਜਾਂਦਾ ਹੈ।
3. ਫਿਰ `get_videos` ਫੰਕਸ਼ਨ ਨੂੰ ਕਵੈਰੀ ਲਈ ਐਮਬੈਡਿੰਗ ਇੰਡੈਕਸ ਵਿੱਚ ਖੋਜ ਕਰਨ ਲਈ ਕਾਲ ਕੀਤਾ ਜਾਂਦਾ ਹੈ।
4. ਆਖਿਰਕਾਰ, `display_results` ਫੰਕਸ਼ਨ ਨੂੰ ਨਤੀਜੇ ਯੂਜ਼ਰ ਨੂੰ ਦਿਖਾਉਣ ਲਈ ਕਾਲ ਕੀਤਾ ਜਾਂਦਾ ਹੈ।
5. ਫਿਰ ਯੂਜ਼ਰ ਨੂੰ ਇੱਕ ਹੋਰ ਕਵੈਰੀ ਦਰਜ ਕਰਨ ਲਈ ਪ੍ਰੰਪਟ ਕੀਤਾ ਜਾਂਦਾ ਹੈ। ਇਹ ਪ੍ਰਕਿਰਿਆ ਜਾਰੀ ਰਹਿੰਦੀ ਹੈ ਜਦ ਤੱਕ ਯੂਜ਼ਰ `exit` ਨਹੀਂ ਦਿੰਦਾ।

![](../../../../translated_images/pa/notebook-search.1e320b9c7fcbb0bc.png)

ਤੁਹਾਨੂੰ ਇੱਕ ਕਵੈਰੀ ਦਰਜ ਕਰਨ ਲਈ ਪ੍ਰੰਪਟ ਕੀਤਾ ਜਾਵੇਗਾ। ਇੱਕ ਕਵੈਰੀ ਦਰਜ ਕਰੋ ਅਤੇ ਐਂਟਰ ਦਬਾਓ। ਐਪਲੀਕੇਸ਼ਨ ਤੁਹਾਨੂੰ ਕਵੈਰੀ ਨਾਲ ਸੰਬੰਧਿਤ ਵੀਡੀਓਜ਼ ਦੀ ਸੂਚੀ ਵਾਪਸ ਕਰੇਗੀ। ਐਪਲੀਕੇਸ਼ਨ ਇਸ ਗੱਲ ਦਾ ਵੀ ਲਿੰਕ ਵਾਪਸ ਕਰੇਗੀ ਕਿ ਵੀਡੀਓ ਵਿੱਚ ਸਵਾਲ ਦਾ ਜਵਾਬ ਕਿੱਥੇ ਹੈ।

ਇੱਥੇ ਕੁਝ ਕਵੈਰੀਆਂ ਹਨ ਜੋ ਤੁਸੀਂ ਕੋਸ਼ਿਸ਼ ਕਰ ਸਕਦੇ ਹੋ:

- ਅਜ਼ੂਰ ਮਸ਼ੀਨ ਲਰਨਿੰਗ ਕੀ ਹੈ?
- ਕਨਵੋਲੂਸ਼ਨਲ ਨਿਊਰਲ ਨੈੱਟਵਰਕ ਕਿਵੇਂ ਕੰਮ ਕਰਦੇ ਹਨ?
- ਨਿਊਰਲ ਨੈੱਟਵਰਕ ਕੀ ਹੈ?
- ਕੀ ਮੈਂ ਅਜ਼ੂਰ ਮਸ਼ੀਨ ਲਰਨਿੰਗ ਨਾਲ ਜੂਪਿਟਰ ਨੋਟਬੁੱਕਸ ਵਰਤ ਸਕਦਾ ਹਾਂ?
- ONNX ਕੀ ਹੈ?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from input
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**ਅਸਵੀਕਾਰੋਪੱਤਰ**:  
ਇਹ ਦਸਤਾਵੇਜ਼ AI ਅਨੁਵਾਦ ਸੇਵਾ [Co-op Translator](https://github.com/Azure/co-op-translator) ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਅਨੁਵਾਦ ਕੀਤਾ ਗਿਆ ਹੈ। ਜਦੋਂ ਕਿ ਅਸੀਂ ਸਹੀਤਾ ਲਈ ਕੋਸ਼ਿਸ਼ ਕਰਦੇ ਹਾਂ, ਕਿਰਪਾ ਕਰਕੇ ਧਿਆਨ ਵਿੱਚ ਰੱਖੋ ਕਿ ਸਵੈਚਾਲਿਤ ਅਨੁਵਾਦਾਂ ਵਿੱਚ ਗਲਤੀਆਂ ਜਾਂ ਅਸਮਰਥਤਾਵਾਂ ਹੋ ਸਕਦੀਆਂ ਹਨ। ਮੂਲ ਦਸਤਾਵੇਜ਼ ਆਪਣੀ ਮੂਲ ਭਾਸ਼ਾ ਵਿੱਚ ਪ੍ਰਮਾਣਿਕ ਸਰੋਤ ਮੰਨਿਆ ਜਾਣਾ ਚਾਹੀਦਾ ਹੈ। ਮਹੱਤਵਪੂਰਨ ਜਾਣਕਾਰੀ ਲਈ, ਪੇਸ਼ੇਵਰ ਮਨੁੱਖੀ ਅਨੁਵਾਦ ਦੀ ਸਿਫਾਰਸ਼ ਕੀਤੀ ਜਾਂਦੀ ਹੈ। ਅਸੀਂ ਇਸ ਅਨੁਵਾਦ ਦੀ ਵਰਤੋਂ ਤੋਂ ਉਤਪੰਨ ਕਿਸੇ ਵੀ ਗਲਤਫਹਿਮੀ ਜਾਂ ਗਲਤ ਵਿਆਖਿਆ ਲਈ ਜ਼ਿੰਮੇਵਾਰ ਨਹੀਂ ਹਾਂ।
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
